In [1]:
pip install quandl

Note: you may need to restart the kernel to use updated packages.


In [2]:
# this program predicts stock prices by using ML algo

In [4]:
# installing dependencies
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [29]:
# getting stock data
df=quandl.get("WIKI/FB")

In [31]:
# we have exported this file to csv
df.to_csv("df.csv")

In [48]:
df=pd.read_csv("df.csv")

In [33]:
import pandas as pd

In [49]:
df.head(1)

,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2012-05-18,42.05,45.0,38.0,38.2318,573576400.0,0.0,1.0,42.05,45.0,38.0,38.2318,573576400.0


In [50]:
# as we want to make date as index 
df=df.set_index("Date")

In [51]:
df.head(1)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2012-05-18,42.05,45.0,38.0,38.2318,573576400.0,0.0,1.0,42.05,45.0,38.0,38.2318,573576400.0


In [52]:
df=df[["Adj. Close"]]
# if we are taking single bracket then the col name is not coming

In [53]:
df.head(5)

,Adj. Close
Date,
2012-05-18,38.2318
2012-05-21,34.0300
2012-05-22,31.0000
2012-05-23,32.0000
2012-05-24,33.0300


In [62]:
# create a new col and putting in this dataset
forecast_out=30
df["Prediction"]=df["Adj. Close"].shift(-1)
df.head(3)

In [55]:
df.tail()

,Adj. Close,Prediction
Date,,
2018-03-21,169.39,164.89
2018-03-22,164.89,159.39
2018-03-23,159.39,160.06
2018-03-26,160.06,152.19
2018-03-27,152.19,NaN


In [56]:
df["Prediction"]=df["Adj. Close"].shift(-2)

In [57]:
df.head(2)

,Adj. Close,Prediction
Date,,
2012-05-18,38.2318,31.0
2012-05-21,34.0300,32.0


In [59]:
df.tail(3)

,Adj. Close,Prediction
Date,,
2018-03-23,159.39,152.19
2018-03-26,160.06,NaN
2018-03-27,152.19,NaN


In [63]:
df["Prediction"]=df["Adj. Close"].shift(-forecast_out)

In [64]:
df.head(5)

,Adj. Close,Prediction
Date,,
2012-05-18,38.2318,30.771
2012-05-21,34.0300,31.200
2012-05-22,31.0000,31.470
2012-05-23,32.0000,31.730
2012-05-24,33.0300,32.170


In [65]:
df.tail()

,Adj. Close,Prediction
Date,,
2018-03-21,169.39,NaN
2018-03-22,164.89,NaN
2018-03-23,159.39,NaN
2018-03-26,160.06,NaN
2018-03-27,152.19,NaN


In [66]:
# create the independent dataset (x)
# convert the dataframe to a numpy array
x=np.array(df.drop(["Prediction"],1))
x=x[:-forecast_out]
x

array([[ 38.2318],
       [ 34.03  ],
       [ 31.    ],
       ...,
       [171.5499],
       [175.98  ],
       [176.41  ]])

In [67]:
# create the dependent dataset y
# convert the dataframe to a numpy array
y=np.array(df["Prediction"])
y

array([30.771, 31.2  , 31.47 , ...,    nan,    nan,    nan])

In [68]:
# get all values except the last n rows
y=y[:-forecast_out]
y

array([ 30.771,  31.2  ,  31.47 , ..., 159.39 , 160.06 , 152.19 ])

In [69]:
# split data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [71]:
# create and train the SVM (regressor)
svr_rbf=SVR(kernel="rbf",C=1e3,gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [72]:
# model testing:score returns the coefficient of determinationr^2 of pred
# the best possible score is 1.0
svm_confidence=svr_rbf.score(x_test,y_test)
print("svm confidence:",svm_confidence)


svm confidence: 0.9865752950421851


In [73]:
# create and train Lin reg model
lr=LinearRegression()
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [74]:
# model testing:score returns the coefficient of determinationr^2 of pred
# the best possible score is 1.0
lr_confidence=lr.score(x_test,y_test)
print("lr confidence:",lr_confidence)

lr confidence: 0.9817047524811504


In [ ]:
# svm method gives good prediction than lin reg

In [76]:
# set x_forecast equal to the last 0 rows of original dataset from adj.close
x_forecast=np.array(df.drop(["Prediction"],1))[-forecast_out:]
x_forecast

array([[173.15],
       [179.52],
       [179.96],
       [177.36],
       [176.01],
       [177.91],
       [178.99],
       [183.29],
       [184.93],
       [181.46],
       [178.32],
       [175.94],
       [176.62],
       [180.4 ],
       [179.78],
       [183.71],
       [182.34],
       [185.23],
       [184.76],
       [181.88],
       [184.19],
       [183.86],
       [185.09],
       [172.56],
       [168.15],
       [169.39],
       [164.89],
       [159.39],
       [160.06],
       [152.19]])

In [77]:
# prediction for next n days using Lin reg
lr_prediction=lr.predict(x_forecast)
print(lr_prediction)

[177.21522996 183.66269135 184.1080419  181.47642501 180.11000854
 182.0331132  183.12624637 187.47853585 189.13847881 185.62628242
 182.44809894 180.03915732 180.72742635 184.55339245 183.92585304
 187.90364319 186.51698352 189.44212692 188.96641155 186.05138976
 188.38948016 188.05546724 189.30042447 176.61805535 172.15442824
 173.40950706 168.85478551 163.28790362 163.96605105 156.00034914]


In [78]:
# prediction for next n days using SVM
svm_prediction=svr_rbf.predict(x_forecast)
print(svm_prediction)

[176.01003938 179.70842708 179.75198914 176.33070105 175.18255866
 177.44534667 179.48019368 186.84828094 179.14847653 183.76799254
 178.39194008 175.1172978  175.57487096 180.11530573 179.72104067
 184.83380636 187.48673767 178.87427817 179.56651984 185.80032074
 182.09607561 183.98478267 178.92755172 178.22202402 172.35550237
 170.9214428  172.21313267 167.54310535 166.28326663 162.92943713]


In [ ]:
# we can also run all these codes after importing data from 

In [79]:
# getting stock data
amzn=quandl.get("WIKI/AMZN")

In [80]:
amzn.head(3)

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1997-05-16,22.38,23.75,20.50,20.75,1225000.0,0.0,1.0,1.865000,1.979167,1.708333,1.729167,14700000.0
1997-05-19,20.50,21.25,19.50,20.50,508900.0,0.0,1.0,1.708333,1.770833,1.625000,1.708333,6106800.0
1997-05-20,20.75,21.00,19.63,19.63,455600.0,0.0,1.0,1.729167,1.750000,1.635833,1.635833,5467200.0


In [ ]:
# repeat all codes on this data